In [ ]:
%pip install pandas

In [ ]:
import pandas as pd
import math

# Open vehicles dataset
ce = pd.read_csv('data/processed/veicoli/categorie_euro_2014_2021.csv')

# Open emissions dataset
se_folder = 'data/processed/emissioni/standards_for_'

# Create a dictionary of dataframes, one for each vehicle type
related_se = {
    'Autovetture': pd.read_csv(f'{se_folder}passenger_cars_petrol.csv'),
    'Motrici': pd.read_csv(f'{se_folder}light_commercial_vehicles_petrol.csv'),
    'Veicoli industriali': pd.read_csv(f'{se_folder}large_goods_vehicles.csv'),
    'Autobus': pd.read_csv(f'{se_folder}trucks_and_buses.csv'),
    'Motocicli': pd.read_csv(f'{se_folder}motor_cycles.csv'),
}

euro_classes = [
    { 'kwh': 55, 'km': 20 },
    { 'kwh': 60, 'km': 20 },
    { 'kwh': 65, 'km': 20 },
    { 'kwh': 70, 'km': 20 },
    { 'kwh': 75, 'km': 40 },
    { 'kwh': 80, 'km': 50 },
    { 'kwh': 85, 'km': 60 },
]

# Iterate over the rows of the vehicles dataset
for index, row in ce.iterrows():
    vtype = row['Tipo Veicolo']
    # Get the corresponding dataframe
    se = related_se[vtype]

    for i in range(7):
        euro_vecs = row[f'Euro {i}']
        kwh = euro_classes[i]['kwh']
        km = euro_classes[i]['km']
        # Get the corresponding row
        se_row = se.loc[se['Tier'] == f'Euro {i}']
        co_cols = se_row.filter(like='CO')
        co_arr = co_cols.iloc[:, 0].values
        # Get the next row
        next_se_row = se.loc[se['Tier'] == f'Euro {i+1}']
        next_co_cols = next_se_row.filter(like='CO')
        next_co_arr = next_co_cols.iloc[:, 0].values
        next_co_value = next_co_arr[0] if len(next_co_arr) > 0 else math.ceil(co_arr[0])
        co_value = co_arr[0] if len(co_arr) > 0 else math.ceil(next_co_value)
        # Get the corresponding emission value
        # [numero veicoli classe euro] x ([CO2 tipo di veicolo] x [kWh classe euro]) x [media km percorsi]
        print(f"Veicoli {vtype}: {euro_vecs}, Euro: {i}, CO: {co_value} | Totale: {math.trunc(euro_vecs * co_value * kwh * km)} g/km")